In [194]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [195]:
# cargamos los datos de dos pestañas diferentes
file = "dataset.xlsx"
df_ind = pd.read_excel(file, sheet_name="Hoja1")
df_dep = pd.read_excel(file, sheet_name="Descenlaces")

# nos quedamos con los 40 pacientes
df_ind = df_ind.head(40)

# definimos las variables independientes
ind_vars = [
    "EDAD",
    "tipo CP",
    "DIA HOSP AL DG",
    "cirugia previa",
    "diálisis",
    "Falla renal",
    "Inmunosupresion",
    "Hosp previa 6m",
    "AKI CON HD durante tto",
    "No Apropiado",
    "Tto apropiado N°1",
    "Tto apropiado N°1 >24hrs",
    "Tto apropiado >48 hrs",
    "Monoterapia 1- cual",
    "combinado A- cual",
    "combinado B-cual",
    "Monoterapia 2- cual",
    "combinado A- cual2",
    "combinado B-cual2",
]

# definimos las variables independientes
dep_vars = [
    "falla clinica",
    "Shock septico durante tto",
    "Falla renal nueva con diálisis",
    "MORTALIDAD 30 DIAS",
    "MORTALIDAD IIH"
]

# definimos una función que mapea los medicamentos a solo 3 categorías
def map_categories(value):
    if 'ceftazidima' in value or 'avi' in value:
        return 'C/A'
    elif value == 'no':
        return 'no'
    else:
        return 'otro'

In [196]:
# empezamos quedandonos solo con las variables relevantes del problema
df_ind = df_ind[ind_vars]
df_dep = df_dep[dep_vars]

# hacemos un unico dataframe con variables indep y dep juntos
df = pd.concat([df_ind, df_dep], axis=1)

# quitamos las filas que tienen algun dato faltante
df = df.dropna(how="any")
df = df.drop([30])
#df = df.drop(30, axis=0) # eliminamos a huenupi
#df = df.drop(29, axis=0)
#df = df.drop(14, axis=0)

In [197]:

# dejamos esta variable como INT ya que ya no está el post mortem
df["DIA HOSP AL DG"] = df["DIA HOSP AL DG"].astype(int)

# quitamos al post-mortem
#df = df[df["DIA HOSP AL DG"] != 108]

# dejamos las categorias en minuscula
for c in df.columns:
    try:
        df[[c]] = df[[c]].apply(lambda col: col.str.lower() if col.dtype == 'object' else col)

    except:
        print("problems with column:", c)

# vamos a quitar los espacios de los datos de "tipo CP"
#df['tipo CP'] = df['tipo CP'].str.strip()

# luego dejamos solo 3 categorias: C/A, otro y no
for c in df.columns[13:-5]:
    df[c] = df[c].apply(map_categories)

# sacamos variables que no queremos
df = df.drop(['No Apropiado', 'Tto apropiado N°1', "tipo CP"], axis=1)

# las categorias binarias las trannsformamos en 0 y 1
label_encoder = LabelEncoder()
for c in df.columns[-5:]:
    df[c] = label_encoder.fit_transform(df[c])

for c in df.columns[2:10]:
    df[c] = label_encoder.fit_transform(df[c])

# las variables continuas las normalizamos al máximo valor
for name in ["EDAD", "DIA HOSP AL DG"]:
    df[name] = df[name]/df[name].max()

# definimos un dataframe para crear un resumen del medicamento
df_ca = pd.DataFrame()
df_ca['medicamento'] = df[df.columns[11:16]].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
df_ca['resumen'] = df_ca["medicamento"].apply(
    lambda x: 'C/A' if 'C/A' in x else ('otro' if 'otro' in x else 'no')
)

# creamos los dummies de el resumen de medicamentos
#dum1 = pd.get_dummies(df["tipo CP"]).astype(int)
dummies = pd.get_dummies(df_ca["resumen"]).astype(int)

#dummies = pd.concat([dum1, dum2], axis=1)

# creamos el X e y para el modelo
objetive = dep_vars[0]
y = df[objetive]
X = pd.concat([df[df.columns[:10]], dummies], axis=1)
#X = X.drop(["tipo CP"], axis=1) # , "kpc", "ndm"

In [198]:
# ----- REGRESION LOGISTICA ----- #

# 1. Dividir el dataset en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# 2. Crear y ajustar el modelo de regresión logística
model = LogisticRegression(solver='liblinear')  # 'liblinear' funciona bien con datasets pequeños
model.fit(X_train, y_train)

# 3. Hacer predicciones sobre los datos de prueba
y_pred = model.predict(X_test)

# 4. Evaluar el rendimiento del modelo
# Calcular la precisión
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')

# Generar la matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(conf_matrix)

# Generar un reporte de clasificación
class_report = classification_report(y_test, y_pred)
print('Classification Report:')
print(class_report)

# 5. Interpretar los coeficientes del modelo
coefficients = model.coef_[0]
importance_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Coefficient': coefficients
}).sort_values(by='Coefficient', ascending=False)

Accuracy: 0.7500
Confusion Matrix:
[[2 0]
 [2 4]]
Classification Report:
              precision    recall  f1-score   support

           0       0.50      1.00      0.67         2
           1       1.00      0.67      0.80         6

    accuracy                           0.75         8
   macro avg       0.75      0.83      0.73         8
weighted avg       0.88      0.75      0.77         8



In [199]:
print('Feature Importances (Coefficients):')
importance_df

Feature Importances (Coefficients):


,Feature,Coefficient
3,diálisis,1.054514
4,Falla renal,1.002344
11,no,0.659518
8,Tto apropiado N°1 >24hrs,0.522916
7,AKI CON HD durante tto,0.452496
12,otro,0.285369
6,Hosp previa 6m,0.183925
2,cirugia previa,0.159393
5,Inmunosupresion,-0.035979
1,DIA HOSP AL DG,-0.224642


In [200]:
coef = importance_df[importance_df["Feature"]=="C/A"]["Coefficient"].values[0]

# Calcula el Odds Ratio
odds_ratio_C_A = np.exp(coef)

In [201]:
print("Odds Ratio:", odds_ratio_C_A)

Odds Ratio: 0.3048154064272619


In [158]:
X[X["C/A"]==1].shape[0], X[X["C/A"]==0].shape[0]

(23, 16)

In [137]:
X.shape

(38, 13)

In [59]:
y

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    1
13    1
14    0
15    0
16    0
17    0
18    0
19    1
20    1
21    1
22    1
23    1
24    1
25    1
26    1
27    1
28    1
29    1
30    1
31    1
32    1
33    1
34    1
35    1
36    1
37    1
38    1
39    1
Name: falla clinica, dtype: int64